# **Credit-Card-Approvals**

## **Overview**

A credit card is a financial tool issued by banks or financial institutions that allows the cardholder to borrow funds to make purchases, pay for services, or withdraw cash up to a certain credit limit. Credit Card Approvals is the process by which a bank or financial institution evaluates a person's credit-worthiness to decide whether or not to approve their application for a credit card.

#### **Data Description**
The dataset comprises two main files, `Credit_card.csv` and `Credit_card_label.csv`, which together provide detailed information about credit card applicants and the outcomes of their applications. This dataset provides a comprehensive overview of the applicant's demographic, financial, and employment information, which can be used to analyze factors affecting credit card approval decisions. Below is a description of each feature in the datasets:

##### 1. **`Credit_card.csv`**
- `Ind_ID`: Unique identifier for each client.
- `Gender`: Gender of the applicant (e.g., Male, Female).
- `Car_owner`: Indicates whether the applicant owns a car (Yes/No).
- `Propert_owner`: Indicates whether the applicant owns property (Yes/No).
- `Children`: The number of children the applicant has.
- `Annual_income`: The annual income of the applicant.
- `Type_Income`: Type of income the applicant receives (e.g., Salary, Pension, Business).
- `Education`: The highest level of education attained by the applicant (e.g., High School, Bachelor’s, Master’s).
- `Marital_status`: The marital status of the applicant (e.g., Single, Married, Divorced).
- `Housing_type`: The applicant’s living situation (e.g., Renting, Homeowner, Living with parents).
- `Birthday_count`: The applicant's age represented as the backward count of days from the current day (0 means today, -1 means yesterday, etc.).
- `Employed_days`: The number of days since the applicant started their current employment, represented as a backward count from the current day (0 means today, positive values indicate the applicant is currently unemployed).
- `Mobile_phone`: Indicates whether the applicant has a mobile phone (Yes/No).
- `Work_phone`: Indicates whether the applicant has a work phone (Yes/No).
- `Phone`: Indicates whether the applicant has any phone number (Yes/No).
- `EMAIL_ID`: Indicates whether the applicant has an email address (Yes/No).
- `Type_Occupation`: The occupation type of the applicant (e.g., Manager, Sales, Laborer).
- `Family_Members`: The size of the applicant's family, including themselves.

##### 2. **`Credit_card_label.csv`**
- `ID`: The unique identifier (Ind_ID) used to join with the application data in the Credit_Card.csv file.
- `Label`: The outcome of the credit card application, where 0 indicates the application was approved, and 1 indicates it was rejected.

<a id="cont"></a>

## Table of Contents

- [1. Import Packages](#one)
- [2. Load Data](#two)
- [3. Exploratory Data Analysis (EDA)](#three)
- [4. Feature Engineering](#four)
- [5. Modeling](#five)
- [6. Model Explanation](#six)
- [7. Model Evaluation](#seven)
- [8. Model Deployment](#eight)
- [9. Conclusion](#nine)

<a id="one"></a>
# 1. Import Packages
[Back to Table of Contents](#cont)

---

In [136]:
import pandas as pd
import numpy as np
import nltk
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import joblib

<a id="two"></a>
# 2. Load Data
[Back to Table of Contents](#cont)

---

In [137]:
data = pd.read_csv("/Users/user/Downloads/dataset/Credit_card.csv")

In [138]:
label = pd.read_csv("/Users/user/Downloads/dataset/Credit_card_label.csv")

<a id="three"></a>
# 3. Exploratory Data Analysis (EDA)
[Back to Table of Contents](#cont)

---

##### The dataset consisting of various background information about the applicant

In [139]:
data.head()

,Ind_ID,GENDER,Car_Owner,Propert_Owner,CHILDREN,Annual_income,Type_Income,EDUCATION,Marital_status,Housing_type,Birthday_count,Employed_days,Mobile_phone,Work_Phone,Phone,EMAIL_ID,Type_Occupation,Family_Members
0,5008827,M,Y,Y,0,180000.0,Pensioner,Higher education,Married,House / apartment,-18772.0,365243,1,0,0,0,NaN,2
1,5009744,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,-13557.0,-586,1,1,1,0,NaN,2
2,5009746,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,NaN,-586,1,1,1,0,NaN,2
3,5009749,F,Y,N,0,NaN,Commercial associate,Higher education,Married,House / apartment,-13557.0,-586,1,1,1,0,NaN,2
4,5009752,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,-13557.0,-586,1,1,1,0,NaN,2


##### The dataset consisting of the labels of whether the credit card application was approved or not

In [140]:
label.head()

,Ind_ID,label
0,5008827,1
1,5009744,1
2,5009746,1
3,5009749,1
4,5009752,1


##### Merge the two dataset files 

In [141]:
data = data.merge(label, on="Ind_ID", how="left")

In [142]:
data.head()

,Ind_ID,GENDER,Car_Owner,Propert_Owner,CHILDREN,Annual_income,Type_Income,EDUCATION,Marital_status,Housing_type,Birthday_count,Employed_days,Mobile_phone,Work_Phone,Phone,EMAIL_ID,Type_Occupation,Family_Members,label
0,5008827,M,Y,Y,0,180000.0,Pensioner,Higher education,Married,House / apartment,-18772.0,365243,1,0,0,0,NaN,2,1
1,5009744,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,-13557.0,-586,1,1,1,0,NaN,2,1
2,5009746,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,NaN,-586,1,1,1,0,NaN,2,1
3,5009749,F,Y,N,0,NaN,Commercial associate,Higher education,Married,House / apartment,-13557.0,-586,1,1,1,0,NaN,2,1
4,5009752,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,-13557.0,-586,1,1,1,0,NaN,2,1


In [143]:
# Display basic information about the data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1548 entries, 0 to 1547
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Ind_ID           1548 non-null   int64  
 1   GENDER           1541 non-null   object 
 2   Car_Owner        1548 non-null   object 
 3   Propert_Owner    1548 non-null   object 
 4   CHILDREN         1548 non-null   int64  
 5   Annual_income    1525 non-null   float64
 6   Type_Income      1548 non-null   object 
 7   EDUCATION        1548 non-null   object 
 8   Marital_status   1548 non-null   object 
 9   Housing_type     1548 non-null   object 
 10  Birthday_count   1526 non-null   float64
 11  Employed_days    1548 non-null   int64  
 12  Mobile_phone     1548 non-null   int64  
 13  Work_Phone       1548 non-null   int64  
 14  Phone            1548 non-null   int64  
 15  EMAIL_ID         1548 non-null   int64  
 16  Type_Occupation  1060 non-null   object 
 17  Family_Members

In [144]:
# Display descriptive statistics
data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Ind_ID,1548.0,5.078920e+06,41717.587742,5008827.0,5045069.75,5078841.5,5115673.00,5150412.0
CHILDREN,1548.0,4.127907e-01,0.776691,0.0,0.00,0.0,1.00,14.0
Annual_income,1525.0,1.913993e+05,113252.997656,33750.0,121500.00,166500.0,225000.00,1575000.0
Birthday_count,1526.0,-1.604034e+04,4229.503202,-24946.0,-19553.00,-15661.5,-12417.00,-7705.0
Employed_days,1548.0,5.936469e+04,137808.062701,-14887.0,-3174.50,-1565.0,-431.75,365243.0
Mobile_phone,1548.0,1.000000e+00,0.000000,1.0,1.00,1.0,1.00,1.0
Work_Phone,1548.0,2.080103e-01,0.406015,0.0,0.00,0.0,0.00,1.0
Phone,1548.0,3.094315e-01,0.462409,0.0,0.00,0.0,1.00,1.0
EMAIL_ID,1548.0,9.237726e-02,0.289651,0.0,0.00,0.0,0.00,1.0
Family_Members,1548.0,2.161499e+00,0.947772,1.0,2.00,2.0,3.00,15.0


In [145]:
# Check for missing values in the data
data.isna().sum()

Ind_ID               0
GENDER               7
Car_Owner            0
Propert_Owner        0
CHILDREN             0
Annual_income       23
Type_Income          0
EDUCATION            0
Marital_status       0
Housing_type         0
Birthday_count      22
Employed_days        0
Mobile_phone         0
Work_Phone           0
Phone                0
EMAIL_ID             0
Type_Occupation    488
Family_Members       0
label                0
dtype: int64

- `Type_Occupation` is the column with the most missing values (488 entries) followed by `Annual_income` with 23 entries, `Birthday_count` with 22 entries, and `Gender` with 7 entries. 

In [146]:
data.drop("Type_Occupation", axis=1, inplace=True)

In [147]:
data.columns

Index(['Ind_ID', 'GENDER', 'Car_Owner', 'Propert_Owner', 'CHILDREN',
       'Annual_income', 'Type_Income', 'EDUCATION', 'Marital_status',
       'Housing_type', 'Birthday_count', 'Employed_days', 'Mobile_phone',
       'Work_Phone', 'Phone', 'EMAIL_ID', 'Family_Members', 'label'],
      dtype='object')

- The `Type_Occupation` column consisting of the type of occupation the applicant does has been successfully dropped due to having a high count of missing values.   

In [148]:
data.dropna(inplace=True)

In [149]:
data.isna().sum()

Ind_ID            0
GENDER            0
Car_Owner         0
Propert_Owner     0
CHILDREN          0
Annual_income     0
Type_Income       0
EDUCATION         0
Marital_status    0
Housing_type      0
Birthday_count    0
Employed_days     0
Mobile_phone      0
Work_Phone        0
Phone             0
EMAIL_ID          0
Family_Members    0
label             0
dtype: int64

- All the rows consisting of missing values in the columns(`Annual_income`, `Birthday_count` and `Gender`) have been successfully dropped to ensure that our dataset remains consistent, accurate and reliable for analysis. 

In [151]:
data.duplicated().sum()

0

- There seems to be no duplicated values in our dataset therefore implies that each entity in our dataset is only represented once.